In [1]:
!pip install transformers[sentencepiece]

In [2]:
import pandas as pd
from transformers import pipeline

classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Specifying our labels

In [3]:
labels = ['bullying', "insult", "profanity", 'sarcasm', 'threat', 'exclusion',
          'porn', 'spam', 'lgbt', 'black lives matter', 'ecological activism',
          "feminism", 'politics', 'positive', 'negative', ]

--------------------

In [4]:
my_tweets = pd.read_csv("tweets_clear.csv")
my_tweets.head()

,username,profile_id,tweet
0,@anitasarkeesian,1265443617981661184,Im really excited to finally be able to share ...
1,@anitasarkeesian,1265443617981661184,Since I get asked a lot here is a list of some...
2,@anitasarkeesian,1265443617981661184,While I have feelings about the nuances of Bar...
3,@anitasarkeesian,1265443617981661184,Im havent been super active on socials these d...
4,@anitasarkeesian,1265443617981661184,While FemFreq is winding down Im still consult...


In [6]:
import pandas as pd
from tqdm import tqdm
def classify_df(tweets_df: pd.DataFrame, labels: list[str], text_column: str='tweet') -> pd.DataFrame:
    """
    Classifying tweets in a dataframe by your labels.

    :param tweets_df: DataFrame containing the tweets.
    :param text_column: Name of the column containing the tweet texts.
    :param labels: List of labels to classify the tweets.
    :return: DataFrame with original tweets and their classification scores.
    """

    # Список для хранения результатов
    results = []

    # Итерируемся по строкам датафрейма
    for index, row in tqdm(tweets_df.iterrows(), total=tweets_df.shape[0], desc="Classifying tweets"):
        tweet = row[text_column]
        result = classifier(tweet, labels)

        # Создаем словарь с результатами для текущего твита
        result_dict = {label: score for label, score in zip(result['labels'], result['scores'])}
        # print("Tweet - ", tweet)
        # print(result_dict)
        result_dict['tweet'] = tweet

        # Добавляем словарь с результатами в список
        results.append(result_dict)

    # Преобразуем список словарей в DataFrame
    results_df = pd.DataFrame(results)

    # Переставляем колонку 'tweet' на первое место
    cols = ['tweet'] + [col for col in results_df if col != 'tweet']
    results_df = results_df[cols]

    return results_df



In [7]:
labeled_data = classify_df(my_tweets, labels)

Classifying tweets: 100%|██████████| 1608/1608 [3:01:28<00:00,  6.77s/it]  


In [9]:
labeled_data.head()

,tweet,positive,exclusion,sarcasm,threat,negative,lgbt,profanity,insult,spam,black lives matter,feminism,ecological activism,porn,politics,bullying
0,Im really excited to finally be able to share ...,0.432244,0.098964,0.078851,0.078805,0.059108,0.040457,0.039513,0.032489,0.026300,0.024117,0.022569,0.018696,0.018097,0.015657,0.014132
1,Since I get asked a lot here is a list of some...,0.113490,0.097633,0.121824,0.191490,0.075059,0.044796,0.044845,0.040130,0.046291,0.070176,0.028421,0.035053,0.029786,0.024716,0.036290
2,While I have feelings about the nuances of Bar...,0.038837,0.171000,0.058777,0.378630,0.137242,0.033674,0.022885,0.065519,0.017054,0.011834,0.014085,0.007217,0.010685,0.015778,0.016783
3,Im havent been super active on socials these d...,0.006929,0.051875,0.676443,0.109638,0.054106,0.019283,0.010143,0.019996,0.014677,0.006276,0.013295,0.005069,0.004059,0.004855,0.003355
4,While FemFreq is winding down Im still consult...,0.070841,0.091553,0.067348,0.099455,0.086908,0.063509,0.037158,0.024493,0.023590,0.027653,0.368441,0.009666,0.009392,0.005954,0.014037


In [8]:
labeled_data.to_csv("labeled_data.csv", index=False)